In [1]:
from azureml.core import Dataset
from azureml.core import Workspace

import pandas as pd
from io import StringIO

# Conecta ao Azure ML Workspace
workspace = Workspace.from_config()

# Registra um dataset
datastore = workspace.get_default_datastore()

# Cria um dataset demonstrativo
data = """Age,income,target
25,50000,0
30,60000,1
35,55000,0
40,65000,1"""

dataset_file = StringIO(data)

# Salvar o dataset localmente e subir para o datastore(armazenamento de dados)
df = pd.read_csv(dataset_file)
df.to_csv("training_data.csv",index=False)

datastore.upload_files(
    ["training_data.csv"],
    target_path="data/",
    overwrite=True
)

dataset = Dataset.Tabular.from_delimited_files(path=(datastore, 'data/training_data.csv'))
dataset = dataset.register(workspace=workspace, name='training_dataset', create_new_version=True)
print("Dataset registered successfully.")

Uploading an estimated of 1 files
Uploading training_data.csv
Uploaded training_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
Dataset registered successfully.


In [ ]:
from azureml.core import ScriptRunConfig, Experiment

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget

# Cria um ambiente customizado
env = Environment(name='custom_environment')
dependencies = CondaDependencies.create(
    conda_packages = ['scikit-learn','pandas','numpy','joblib'],
    pip_packages = ['azume-sdk'] # Adiciona um pip se necessário
    )
env.python.conda_dependencies = dependencies

# Registrar ambiente virtual (environment)
env.register(workspace=workspace)
print("Enviroment registered successfully")

# Define a instância computacional
compute_target = ComputeTarget(workspace=workspace, name='D4S-V3-2024-10-30-a')

# Define treinamento, script e configuração
# Especifica diretório contendo o script de treinamento e instância em que será treinado
# O objeto ScriptRunConfig é usado pra encapsular as configurações de treinamento, incluindo argumentos de script
src=ScriptRunConfig(
    source_directory='scripts',
    script='2_lesson_train.py',
    compute_target=compute_target,
    arguments=['--input_data', dataset.as_named_input('training_data')],
    environment=env
)

# Passa o dataset como um argumento para o script

# Submete o experimento
# Um 'Experiment' em Azura ML é um container para rodar todos seus treinamentos
experiment = Experiment(workspace, name='training_experiment')
run = experiment.submit(config=src) # Submete a execução do experimento no computador alvo especificado
run.wait_for_completion(show_output=True) # Espera a conclusão do treinamento para mostrar a saída